In [61]:
#Packages:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import matplotlib.pyplot as plt
import scipy as sc
import os

In [62]:
#Set directory:
os.chdir("C:/Users/CASA2002/Documents/Universidad de los Andes/Big Data/PS3/dataPS3")
tr=pyr.read_r("train.Rds") 
tt=pyr.read_r("test.Rds")
print(tr.keys())
print(tt.keys())
df_tr=tr[None] #Train Data frame (Bogotá D.C. y Medellín). 
df_tt=tt[None] #Test Data frame (Cali). 

odict_keys([None])
odict_keys([None])


In [63]:
#Missing values count/share in train.
df_tr.isnull().sum() 
df_tr.isnull().sum()/len(df_tr)
df_tr.drop("rooms", axis=1, inplace=True) #Correct bedrooms data.

In [ ]:
#Descriptive statistics:
ds=(df_tr[["area", "bedrooms", "bathrooms"]].describe(include="all"))
ds=ds.T
ds=ds[["count", "mean", "std", "min", "50%", "max"]]
ds=ds.round(2)
print(ds.to_latex())